In [1]:
library(pracma)
library(matrixcalc)
library(fields)
library(wordspace)
library(Matrix)
#load source file
src.path <- "../../src"

source(paste(src.path, "DataGenerationProcess", "synth_basis.R", sep="/"))
source(paste(src.path, "DataGenerationProcess", "synth_data.R", sep="/"))
source(paste(src.path, "DataGenerationProcess", "synth_graph.R", sep="/"))
source(paste(src.path, "DataGenerationProcess", "synth_linearop.R", sep="/"))

source(paste(src.path, "Estimation", "basis_estimation.R", sep="/"))
source(paste(src.path, "Utility", "utility.R", sep="/"))
source(paste(src.path, "Estimation", "cca_estimation.R", sep="/"))
source(paste(src.path, "Utility", "R2python.R", sep="/"))



Loading required package: spam

Loading required package: dotCall64

Loading required package: grid

Spam version 2.7-0 (2021-06-25) is loaded.
Type 'help( Spam)' or 'demo( spam)' for a short introduction 
and overview of this package.
Help for individual functions is also obtained by adding the
suffix '.spam' to the function name, e.g. 'help( chol.spam)'.


Attaching package: 'spam'


The following objects are masked from 'package:base':

    backsolve, forwardsolve


Loading required package: viridis

Loading required package: viridisLite


Try help(fields) to get started.

Loading required package: Matrix


Attaching package: 'Matrix'


The following object is masked from 'package:spam':

    det


The following objects are masked from 'package:pracma':

    expm, lu, tril, triu


Loading required package: splines

Loading required package: fds

Loading required package: rainbow

Loading required package: MASS

Loading required package: pcaPP

Loading required package: RCurl

Loadin

In [90]:

cov_name <- 'power'
path <- './'

###specify parameters
n <- 200
p <- 150

M <- 2
obs.time <- seq(0,1,1/50)
k.gen <- 9

m1_arr <- matrix(0,nrow=10,ncol=8)
m2_arr <- matrix(0,nrow=10,ncol=8)
m_coor <- matrix(0,nrow=10,ncol=17)
for (iter in seq(1:10)){
for (n in c(100)){
    for (p in c(150)){
        ## be careful for the choice of the number of basis function 
## fourier basis: km must be odd
## bspline basis km>4
print(paste("N=",n,"p=",p))
km.gen <- c(k.gen,k.gen)

Apinv_list <- list()
A_list <- list()
N_list <- list()
basis.m_list <- list()
true.basis_list <- list()
true.values_list <- list()

#generate data from the graph

if (cov_name == "random"){
    omega <- synth.omega.random(p, k.gen, drop=0.1)
}
if (cov_name == "power"){
    omega <- synth.omega.power(p, k.gen)
    omega <- solve(omega)
}
if (cov_name == "tridiag1"){
    omega <- synth.omega.tridiag1(p, k.gen)
}
if (cov_name == "tridiag2"){
    omega <- synth.omega.tridiag2(p, k.gen)
}
if (cov_name == "tridiag3"){
    omega <- synth.omega.tridiag3(p, k.gen)
}
if (cov_name == "tridiag4"){
    omega <- synth.omega.tridiag4(p, k.gen)

}

#ensure that the diagonal values are all 1

G.true <- matrix(0, p, p) # p by p adjacency matrix
for(i in 1:p){
  for(j in 1:p){
    if(sum(abs(omega[((i-1)*k.gen+1):(i*k.gen), ((j-1)*k.gen+1):(j*k.gen)])) > 0)
      G.true[i,j] <- 1
  }
}

cov <- solve(omega)


for(m in 1:M){
    Am <- synth.linear_op.sparse_orthogonal(k.gen, km.gen[m], k.gen, scale=.2)
    Am <- t(t(Am) %*% diag(.2*((1:k.gen))+1)) #this is to make the singular values distinct

    A_list[[m]] <- Am
    Apinv_list[[m]] <- solve(Am)
}

for(m in 1:M){
    
    #noise covariance
    N_list[[m]] <- diag(p*km.gen[m])*.05
    basis.m_list[[m]] <- synth.fourier.bases.m(obs.time, km.gen[m])
}

#convert to regression B
B_list <- utility.graph2B(omega,p)
# save true graphs

#generate data 
data <- synth.data_from_graph(n, p, cov, basis.m_list, Apinv_list,N_list, dependent=TRUE, addnoise=FALSE)

##estimate A by CCA

#data modality 1

concate_data <- list()
for(m in 1:M){
        dm <- data[[m]][,1,]
    for(i in 2:p){
        dm <- rbind(dm, data[[m]][,p,])
    }
    concate_data[[m]] <- dm
}

#implement basis number selection

for(m in 1:M){
    sum_list <- list()
    j <- 1
    for( b in c(3, 5, 7, 9, 11, 13, 15, 17)){
        
        basis <- create.fourier.basis(rangeval=c(0, 1), nbasis=b)
        sum.a <- 0
        g.fdobj<-Data2fd(argvals=obs.time, t(concate_data[[m]]), basis)

        for(i in 1:(dim(concate_data[[m]])[1])){
            a <- sum(sqrt((eval.fd(obs.time, g.fdobj[i])- concate_data[[m]][i,])**2))/sum(concate_data[[m]][i,]**2)
            sum.a <- sum.a+a
        }
        
        sum_list <- append(sum_list,sum.a)
    
    if (m==1){
       m1_arr[iter,j] <- sum.a
    }
    else{
      m2_arr[iter,j] <- sum.a
    }
    j <- j+1
    }
}


fourier.basis1 <- create.fourier.basis(rangeval=c(0,1), nbasis=17)
d1 <-Data2fd(argvals=obs.time, y=t(data[[1]][,1,]), basisobj=fourier.basis1)
#data modality 2
fourier.basis2 <- create.fourier.basis(rangeval=c(0,1), nbasis=17)
d2 <-Data2fd(argvals=obs.time, y=t(data[[2]][,1,]), basisobj=fourier.basis2)
cca.r.est <- estimate.cca.basis_expansion(d1, d2, 9)

m_coor[iter,] <- cca.r.est$cca
}
}
}


[1] "N= 100 p= 150"


Warning message in rmvnorm(n, rep(0, kp), cov):
"sigma is numerically not positive semidefinite"


[1] "Generating data from modality 1"
[1] "Generating data from modality 2"
[1] "canonical correlation values:"
 [1]  9.989134e-01  9.986811e-01  9.982118e-01  9.976126e-01  9.956980e-01
 [6]  9.939287e-01  9.926462e-01  9.914920e-01  9.886481e-01  1.985621e-15
[11]  1.947099e-17  1.373227e-17  9.971689e-18 -1.900957e-18 -8.161180e-18
[16] -1.231559e-17 -6.663536e-16
[1] "N= 100 p= 150"


Warning message in rmvnorm(n, rep(0, kp), cov):
"sigma is numerically not positive semidefinite"


[1] "Generating data from modality 1"
[1] "Generating data from modality 2"
[1] "canonical correlation values:"
 [1]  9.989134e-01  9.986811e-01  9.982118e-01  9.976126e-01  9.956980e-01
 [6]  9.939287e-01  9.926462e-01  9.914920e-01  9.886481e-01  1.985621e-15
[11]  1.947099e-17  1.373227e-17  9.971689e-18 -1.900957e-18 -8.161180e-18
[16] -1.231559e-17 -6.663536e-16
[1] "N= 100 p= 150"


...

[1] "Generating data from modality 1"
[1] "Generating data from modality 2"
[1] "canonical correlation values:"
 [1]  9.989134e-01  9.986811e-01  9.982118e-01  9.976126e-01  9.956980e-01
 [6]  9.939287e-01  9.926462e-01  9.914920e-01  9.886481e-01  1.985621e-15
[11]  1.947099e-17  1.373227e-17  9.971689e-18 -1.900957e-18 -8.161180e-18
[16] -1.231559e-17 -6.663536e-16
[1] "N= 100 p= 150"


Warning message in rmvnorm(n, rep(0, kp), cov):
"sigma is numerically not positive semidefinite"


[1] "Generating data from modality 1"
[1] "Generating data from modality 2"
[1] "canonical correlation values:"
 [1]  9.989134e-01  9.986811e-01  9.982118e-01  9.976126e-01  9.956980e-01
 [6]  9.939287e-01  9.926462e-01  9.914920e-01  9.886481e-01  1.985621e-15
[11]  1.947099e-17  1.373227e-17  9.971689e-18 -1.900957e-18 -8.161180e-18
[16] -1.231559e-17 -6.663536e-16
[1] "N= 100 p= 150"


Warning message in rmvnorm(n, rep(0, kp), cov):
"sigma is numerically not positive semidefinite"


[1] "Generating data from modality 1"
[1] "Generating data from modality 2"
[1] "canonical correlation values:"
 [1]  9.989134e-01  9.986811e-01  9.982118e-01  9.976126e-01  9.956980e-01
 [6]  9.939287e-01  9.926462e-01  9.914920e-01  9.886481e-01  1.985621e-15
[11]  1.947099e-17  1.373227e-17  9.971689e-18 -1.900957e-18 -8.161180e-18
[16] -1.231559e-17 -6.663536e-16


In [91]:
mean_coor <- matrix(0, ncol=17)
std_coor  <- matrix(0, ncol=17)
for(i in seq(1:17)){
    mean_coor[i] <- mean(m_coor[,i])
    std_coor[i] <- std(m_coor[,i])
}

In [92]:
 mean_coor


0.9989134,0.9986811,0.9982118,0.9976126,0.995698,0.9939287,0.9926462,0.991492,0.9886481,1.985621e-15,1.947099e-17,1.373227e-17,9.971689e-18,-1.900957e-18,-8.16118e-18,-1.231559e-17,-6.663536e-16


In [93]:
 std_coor

0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [94]:
mean(m1_arr)
mean_m1 <- matrix(0,ncol=8)
std_m1  <- matrix(0,ncol=8)
for(i in seq(1:8)){
    mean_m1[i] <- mean(m1_arr[,i])
    std_m1[i]  <- std(m1_arr[,i])
}


mean(m2_arr)
mean_m2 <- matrix(0,ncol=8)
std_m2  <- matrix(0,ncol=8)
for(i in seq(1:8)){
    mean_m2[i] <- mean(m2_arr[,i])
    std_m2[i]  <- std(m2_arr[,i])
}

[1] 1490.016

[1] 1548.258

In [95]:
mean_m1
std_m1

4806.577,3556.196,2233.537,264.6095,264.5661,264.5493,265.5409,264.5517


0,0,0,0,0,0,0,0


In [96]:
mean_m2
std_m2

4989.896,3741.406,2289.805,272.7046,272.7023,272.6796,274.1836,272.6889


0,0,0,0,0,0,0,0
